In [1]:
pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 788.3 kB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 932.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import pandas as pd
from codecarbon import OfflineEmissionsTracker

2024-04-22 13:27:29.204591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 13:27:46.781851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load li

In [3]:
training_set=tf.data.Dataset.load('gs://plant_buddy2/oxford_dataset/training_set')
validation_set=tf.data.Dataset.load('gs://plant_buddy2/oxford_dataset/validation_set')
test_set=tf.data.Dataset.load('gs://plant_buddy2/oxford_dataset/test_set')

training_class_counts = {}
validation_class_counts = {}
test_class_counts = {}

for images, labels in training_set:
    for label in labels.numpy():
        label = label.item()  
        if label in training_class_counts:
            training_class_counts[label] += 1
        else:
            training_class_counts[label] = 1


for images, labels in validation_set:
    for label in labels.numpy():
        label = label.item()  
        if label in validation_class_counts:
            validation_class_counts[label] += 1
        else:
            validation_class_counts[label] = 1


for images, labels in test_set:
    for label in labels.numpy():
        label = label.item() 
        if label in test_class_counts:
            test_class_counts[label] += 1
        else:
            test_class_counts[label] = 1

In [4]:

print("Training set labels:")
for label, count in training_class_counts.items():
    print("{}".format(label))

print("Training set:")
for label, count in training_class_counts.items():
    print("{}".format(count))

print("\nValidation set labels:")
for label, count in validation_class_counts.items():
    print("{}".format(label))

print("\nValidation set:")
for label, count in validation_class_counts.items():
    print("{}".format(count))

print("\nTest set labels:")
for label, count in test_class_counts.items():
    print("{}".format(label))

print("\nTest set:")
for label, count in test_class_counts.items():
    print("{}".format(count))

Training set labels:
25
100
66
17
72
95
43
26
93
88
23
11
90
28
59
64
68
55
69
81
78
70
6
14
12
19
27
3
76
22
56
75
42
74
5
94
7
33
38
73
34
35
18
80
29
97
84
60
82
63
2
86
65
15
87
20
47
50
4
46
45
89
21
0
36
52
49
61
57
24
16
99
85
54
48
71
40
79
83
1
96
92
101
10
77
91
32
41
98
8
62
51
53
67
31
58
39
30
13
37
44
9
Training set:
27
46
30
62
166
70
77
26
143
158
31
73
60
61
90
86
41
86
46
94
27
64
27
36
35
43
49
44
217
74
53
89
116
101
30
107
69
29
29
151
33
57
34
137
67
68
46
32
106
38
29
50
49
31
131
28
57
228
53
54
167
64
46
28
89
73
76
45
97
29
66
33
46
54
37
76
106
88
70
47
49
32
38
70
118
50
33
44
51
33
41
68
48
41
35
52
51
36
32
42
28
32

Validation set labels:
44
8
65
93
92
5
40
85
63
51
46
39
38
21
27
47
99
75
101
37
96
69
11
72
48
74
14
55
57
79
42
76
41
36
54
32
71
94
1
91
53
80
70
50
56
43
34
26
31
3
2
60
81
16
12
78
13
23
62
82
18
68
17
29
49
4
24
59
77
90
100
15
19
6
45
33
35
95
30
84
67
83
97
87
58
52
20
9
28
98
0
22
61
86
88
10
89
64
7
73
66
25

Validation set:
9
9
6
8

In [5]:

num_training_examples = 0
num_validation_examples = 0
num_test_examples = 0

for example in training_set:
  num_training_examples += 1

for example in validation_set:
  num_validation_examples += 1

for example in test_set:
  num_test_examples += 1

print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {}'.format(num_validation_examples))
print('Total Number of Test Images: {} \n'.format(num_test_examples))

num_classes = 102
print('Total Number of Classes: {}'.format(num_classes))


# In[4]:


num_training_batches = tf.data.experimental.cardinality(training_set).numpy()
num_validation_batches = tf.data.experimental.cardinality(validation_set).numpy()
num_test_batches = tf.data.experimental.cardinality(test_set).numpy()

print("Number of batches in the training set:", num_training_batches)
print("Number of batches in the validation set:", num_validation_batches)
print("Number of batches in the test set:", num_test_batches)



Total Number of Training Images: 205
Total Number of Validation Images: 26
Total Number of Test Images: 26 

Total Number of Classes: 102
Number of batches in the training set: 205
Number of batches in the validation set: 26
Number of batches in the test set: 26


In [6]:
input_shape = (224, 224, 3)
IMAGE_RES = 224
BATCH_SIZE = 32

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])


from tensorflow.keras import layers

URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

feature_extractor.trainable = False

model = tf.keras.Sequential([
  data_augmentation,
  feature_extractor,
  layers.Dense(num_classes, activation='softmax')
])


model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=600, output_dir="gs://plant_buddy2/energy", output_file="emissions_rest_apr.csv")
tracker.start()

history = model.fit(training_set,
        batch_size = 32,
        epochs=35,
        validation_data=validation_set)

emissions: float = tracker.stop()
print(emissions)

model.save('gs://plant_buddy2/models/rest_apr')
pd.DataFrame.from_dict(history.history).to_csv('gs://plant_buddy2/histories/history_rest_apr.csv',index=False)

In [ ]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=60, output_dir="gs://plant_buddy2/energy", output_file="emissions_rest_apr_eval.csv")
tracker.start()

loss, accuracy = model.evaluate(test_set)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

emissions: float = tracker.stop()
print(emissions)